<a href="https://colab.research.google.com/github/narayan-bhattarai/AI-Classification/blob/main/GastroVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Path to your ZIP file in Drive (update the path if needed)
zip_path = "/content/drive/MyDrive/GastroVision.zip"

import zipfile
import os

extract_path = "/content/GastroVision"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction completed!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extraction completed!


In [1]:
!pip install torch torchvision scikit-learn grad-cam


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.5-py3-none-any.whl size=44284 sha256=3a451435133648aabbba3b80e832c59367dad194dc40a9e0b51ab10822cf7601
  Stored in directory: /root/.cache/pip/wheels/fb/3b/09/2afc520f3d69bc26ae6bd87416759c820a3f7d05c1a077bbf6
Successfully built grad-cam


In [2]:
from torchvision import transforms

train_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [14]:
import os
DATA_DIR = "/content/GastroVision/GastroVision"
os.listdir(DATA_DIR)


['Colon polyps',
 'Normal esophagus',
 'Normal mucosa and vascular pattern in the large bowel',
 'Erythema']

In [ ]:
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
import numpy as np

full_dataset = ImageFolder(DATA_DIR, transform=train_tfms)
targets = full_dataset.targets
indices = np.arange(len(full_dataset))

train_idx, val_idx = train_test_split(
    indices, test_size=0.2, stratify=targets, random_state=42
)

train_ds = Subset(full_dataset, train_idx)
val_ds = Subset(full_dataset, val_idx)
val_ds.dataset.transform = val_tfms  # apply val transforms

print("Classes:", full_dataset.classes)


In [ ]:
from torch.utils.data import WeightedRandomSampler
from collections import Counter

train_targets = [targets[i] for i in train_idx]
class_counts = Counter(train_targets)

sample_weights = [1.0 / class_counts[t] for t in train_targets]
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

train_loader = DataLoader(train_ds, batch_size=32, sampler=sampler)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights = MobileNet_V3_Large_Weights.IMAGENET1K_V1
model = mobilenet_v3_large(weights=weights)

in_features = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(in_features, len(full_dataset.classes))

model = model.to(device)


In [ ]:
class_weights = torch.tensor(
    [1.0 / class_counts[i] for i in range(len(class_counts))],
    dtype=torch.float
).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
from tqdm import tqdm

def train_epoch():
    model.train()
    total_loss = 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

def eval_epoch():
    model.eval()
    total_loss = 0
    correct = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()

    accuracy = correct / len(val_ds)
    return total_loss / len(val_loader), accuracy


In [ ]:
best_acc = 0

for epoch in range(1, 21):
    train_loss = train_epoch()
    val_loss, val_acc = eval_epoch()

    print(f"Epoch {epoch}: Train={train_loss:.4f} | Val={val_loss:.4f} | Acc={val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("🔥 Saved new best model!")


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        preds = outputs.argmax(1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

print(confusion_matrix(all_labels, all_preds))
print(classification_report(all_labels, all_preds, target_names=full_dataset.classes))
